In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
from tqdm.notebook import tqdm_notebook
import torchvision.datasets as datasets
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchsummary import summary
import time
from torch.autograd import Variable
from pathlib import Path
import shutil, os
from tqdm import tqdm

In [3]:
df = pd.read_csv(r"C:\Users\ragha\Documents\NVIDIA Internship\final_data_path.csv")
df.head()

,lms,faces
0,/media/raghav/spectrograms/id10003,/media/raghav/faces/Aamir_Khan
1,/media/raghav/spectrograms/id10004,/media/raghav/faces/Aaron_Tveit
2,/media/raghav/spectrograms/id10005,/media/raghav/faces/Aaron_Yoo
3,/media/raghav/spectrograms/id10007,/media/raghav/faces/Abigail_Breslin
4,/media/raghav/spectrograms/id10008,/media/raghav/faces/Abigail_Spencer


In [4]:
device = torch.device("cuda")
device

device(type='cuda', index=0)

In [ ]:
'''# CREATE FACE DATASET FROM THE DATA GIVEN FOR THE FACIAL RECOGNITION MODEL
for i in range(len(df)):
    celeb = df['faces'][i].split("/")[4]
    celeb_source = "C:/Users/ragha/Documents/NVIDIA Internship/VGG_ALL_FRONTAL/" + celeb + str('/')
    celeb_dest = 'C:/Users/ragha/Documents/NVIDIA Internship/faces_data/' + celeb
    celeb_source_path = Path(celeb_source)
    os.makedirs(celeb_dest)
    files_in_celeb = celeb_source_path.iterdir()
    for item in files_in_celeb:
        shutil.copy(str(item), celeb_dest)'''

In [7]:
transform1 = transforms.Compose([transforms.ToTensor()])
data = datasets.ImageFolder(r"C:\Users\ragha\Documents\NVIDIA Internship\faces_data", transform = transform1)
train_split = int(np.round(0.79*len(data), decimals=0))
val_split = int(np.round(0.01*len(data), decimals=0))
test_split = int(np.round(0.2*len(data), decimals=0))
train_data, val_data, test_data = torch.utils.data.random_split(data, [train_split,val_split,test_split])

In [8]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data.dataset,batch_size=batch_size,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data.dataset,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data.dataset,batch_size=batch_size,shuffle=True)

In [ ]:
class Face_Detector(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.conv3 = nn.Conv2d(128, 256, 5) 
        self.conv4 = nn.Conv2d(256, 64, 5)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(43264, 1225)        
        
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool(self.act1(self.conv2(x)))
        x = self.act1(self.conv3(x))
        x = self.pool(self.act1(self.conv4(x)))
        x = self.flat(x)
        x = self.fc1(x)

        return x
    
face_detector = Face_Detector().to(device)
face_detector

In [ ]:
optimizer = torch.optim.Adam(face_detector.parameters(), lr = 0.001)
loss_function = nn.CrossEntropyLoss()

In [ ]:
def train(face_detector):
    
    img_size = 128
    epochs = 300
    batch_size = 64
    
    train_loss = []
    train_acc = []
    valid_loss = []
    valid_acc = []
    count = 0

    last_loss = 1000
    patience = 2
    triggertimes = 0

    for iteration in range(epochs):

        start_time = time.time()
        train_batch_loss = []
        train_batch_acc = []
        valid_batch_loss = []
        valid_batch_acc = []
        
        for batch_X, batch_y in tqdm(train_loader):
            
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            face_detector.zero_grad()

            outputs = face_detector(batch_X)
            outputs = outputs.to(device)
            
            matches  = [torch.argmax(x)==y for x, y in zip(outputs, batch_y)]
            train_batch_acc.append(matches.count(True)/len(matches))
            
            loss = loss_function(outputs, batch_y)
            train_batch_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            
        with torch.no_grad():
            for batch_X, batch_y in tqdm(val_loader):
            
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                outputs = face_detector(batch_X)
                outputs = outputs.to(device)

                matches  = [torch.argmax(x)==y for x, y in zip(outputs, batch_y)]
                valid_batch_acc.append(matches.count(True)/len(matches))

                loss = loss_function(outputs, batch_y)
                valid_batch_loss.append(loss.item())
                
        end_time = time.time()
        time_taken = end_time-start_time
        
        train_loss.append(sum(train_batch_loss)/len(train_batch_loss))
        train_acc.append((sum(train_batch_acc)/len(train_batch_acc))*100)
        valid_loss.append(sum(valid_batch_loss)/len(valid_batch_loss))
        valid_acc.append((sum(valid_batch_acc)/len(valid_batch_acc))*100)

        current_loss = sum(valid_batch_loss)/len(valid_batch_loss)

        if current_loss > last_loss:
            trigger_times += 1
            print('Trigger Times:', trigger_times)

            if trigger_times >= patience:
                print('Early stopping!')
                return train_loss, valid_loss, train_acc, valid_acc
                
        else:
            trigger_times = 0

        last_loss = current_loss

        print("Epoch -", count+1)
        print("Time taken -", time_taken, "seconds")
        print("Train Loss -", train_loss[count], "---- Train Accuracy -", train_acc[count])
        print("Validation Loss -", valid_loss[count], "---- Validation Accuracy -", valid_acc[count], "\n")

        torch.save({
            'epoch': iteration+1,
            'model_state_dict': face_detector.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss[count],
            }, '/content/drive/MyDrive')
        
        count = count+1
        
    return train_loss, valid_loss, train_acc, valid_acc
        
train_loss, valid_loss, train_acc, valid_acc = train(face_detector)

In [ ]:
torch.save(face_detector.state_dict(), '/content/drive/MyDrive/face_detector_model.pt')

In [ ]:
test_batch_acc = []

with torch.no_grad():
    for batch_X, batch_y in tqdm(test_loader):

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        outputs = face_detector(batch_X)
        outputs = outputs.to(device)

        matches  = [torch.argmax(x)==y for x, y in zip(outputs, batch_y)]
        test_batch_acc.append(matches.count(True)/len(matches))
        
print("Test accuracy -", (sum(test_batch_acc)/len(test_batch_acc))*100)